# Objetivo posible:

https://upload.wikimedia.org/wikipedia/commons/a/ac/Skulptur_tuebingen_vulva.jpg

# Estilos Posibles a transferir a las imagenes

https://upload.wikimedia.org/wikipedia/commons/4/46/Vincent_Willem_van_Gogh_127.jpg

https://upload.wikimedia.org/wikipedia/commons/7/71/Paul_Gauguin_104.jpg

https://upload.wikimedia.org/wikipedia/commons/6/6a/Vincent_van_Gogh_-_The_yellow_house_%28%60The_street%27%29_-_Google_Art_Project.jpg

https://upload.wikimedia.org/wikipedia/commons/c/ca/Vincent_van_Gogh_-_Sunflowers_%28Metropolitan_Museum_of_Art%29.jpg

https://upload.wikimedia.org/wikipedia/commons/8/8d/Two_sunflowers.jpg





# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path_img = '/content/drive/My Drive/ITBA/01 - TRABAJO_FINAL/img/GOPR0154.JPG'
path_Style = '/content/drive/My Drive/ITBA/01 - TRABAJO_FINAL/style/Two_sunflowers.jpg'

In [0]:
# Imagen para estilo

# Cambios en imagen de Estilo
# !wget https://upload.wikimedia.org/wikipedia/commons/8/8d/Two_sunflowers.jpg

# Original
 !wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Cambios en imagen imagen de contenido
# !wget https://upload.wikimedia.org/wikipedia/commons/a/ac/Skulptur_tuebingen_vulva.jpg

# Original
 !wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2020-02-25 19:53:42--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K   722KB/s    in 0.3s    

2020-02-25 19:53:43 (722 KB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2020-02-25 19:53:44--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [imag

In [0]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

Using TensorFlow backend.


In [0]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

# Modificacion a Contexto
# base_image_path = Path('/content/Skulptur_tuebingen_vulva.jpg')

# Original
 base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")

# Modificacion a Estilo
# style_reference_image_path = Path('/content/Two_sunflowers.jpg')

# Original
 style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")


result_prefix = Path("/content/output")
iterations = 40

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

*Respuesta:* 

<font color="003300"><B>
**total_variation_weight:**</b>

- Se aplica en la búsqueda de equilibrio entre la estilización de la imagen y efectos de estilo.

- Permite balancear el peso que tendrá en la Loss Total la función total_variation_loss(combination_image). Logrando que el cómputo de total_variation_loss(combination_image), el cual es creciente por cada ejecución de bucle, reduzca la convergencia de la loss, este proceso tiene efecto sobre la prevalencia de las capas intermedias, haciendo que los contornos definidos por la función de contexto tiendan a desaparecer y sean exaltados los contornos propios existentes en la imagen de estilo cuando tiende a crecer. 

- A continuación se expone la resultante de haber valorado, para la primera imagen, la variable total_variation_weight con una alta ponderación 0.9, a su lado se presenta la segunda con tendencia a neutral a cero, es de notar como en la segunda imagen los contornos mas definidos.
</font>


| total_variation_weight = 0.9 | total_variation_weight = 0.1        
|:-: | :-:
| <img src="image/Res_Loss_stylw_1_contw_10_TotVar_0.9_ite_39.png">| <img src="image/Res_Loss_stylw_1_contw_10_TotVar_0_iter_39.png">

<font color="003300"><b>
**style_weight**</b>

- Se utiliza para definir el parámetro BETA, el cual define el peso en la Loss de la imagen de Estilo. El estilo se mide como la correlación entre diferentes activaciones del mapa de features en una capa particular. Cuando el estilo cobra peso en el balance de la ecuación de la loss, los detalles de los contornos de los objetos y los objetos de menor participación en la imagen total serán reconocidos con menor facilidad.
</font>

| style_weight = 1 | style_weight = 10        
|:-: | :-:
| <img src="image/Res_Loss_stylw_1_contw_10_ite_39.png">| <img src="image/Res_Loss_stylw_10_contw_1_ite_39.png">

<font color="003300"><b>
**content_weight:**</b>

-  Se utiliza para definir el parámetro ALFA, define la relevancia de la Loss para la imagen de Contexto. Las capas superiores de una CNN que ha sido entrenada con un gran conjunto de datos, como ser la (VGG19), aprende a identificar explícitamente las representaciones de objetos que estén contenidos en la imagen, la particularidad de este aprendizaje pone en evidencia que el aprendizaje de valores de pixeles no se da de esta manera, es por ello una imagen que genera una activación similar a otra en una capa superior tendrá contenido similar pero es más difícil de garantizar resultado similar en los pixels.
</font>

| content_weight = 1 | content_weight = 10        
|:-: | :-:
| <img src="image/Res_Loss_stylw_10_contw_1_ite_39.png">| <img src="image/Res_Loss_stylw_1_contw_10_ite_39.png">

<font color="003300">
    
- Los efectos más apreciables en los cambios de pesos de los ponderadores de Estilo y Contexto se hacen más visibles al observar el reflejo de las figuras sobre el agua, donde a las capas superiores comienzan a recibir clasificaciones no tan probables para determinados objetos. En los casos donde el estilo toma preponderancia por sobre el contexto, las líneas que definen el contorno de los objetos como así también los objetos que la conforman tienden a desvanecerse. A diferencia de los cambios de contorno expuestos por modificación del peso del total_variation_weight, elevar la valoración de estilo hace que la pérdida de contorno esté dada por la aparición de los contornos existentes en la imagen de estilo.

- La estandarización de las imágenes (definición de tamaño) se realiza a través de la combinación de los parámetros en la celda en la que se definen los valores de img_nrows y img_ncols siendo:  width, height = load_img(base_image_path).size de donde se obtienen de la imagen de contexto los valores de ancho y alto de la imagen que aporta el contexto para la imagen final. Estos valores son aplicados en la determinación de los valores de la variables img_ncols: img_ncols=int(width*img_nrows/height), la que a su vez adopta el parámetro definido en img_nrows

- Es decir que esta celda aporta los valores de ancho y alto de la imagen de Contexto para posteriormente determinar los valores de las variables img_nrows y img_nclos, las que serán utilizadas durante el proceso de convergencia de las imágenes de estilo y contexto.
</font>





In [0]:
total_variation_weight = 0.9          # Se aplica en la búsqueda de equilibrio entre la estilización de la imagen y efectos de estilo
style_weight = 1                      # Beta que da el peso del stilo en la loss
content_weight = 10                   # Alfa que da el peso del contexto en la loss


#Celda incorporada para clasificar las salidas en funcion de los parámetros elegidos

NombreSalida = '_Loss_stylw_' + str(style_weight) + '_contw_' + str(content_weight) + '_TotVar_' + str(total_variation_weight)

In [0]:
                                      # Definimos el tamaño de las imágenes a utilizar / generar al aplicar las dimensiones en el tensor que incorpora combination_image
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

*Respuesta:*
<font color="003300">
**preprocess_imege()**

- La funcion  tiene por objetivo abrir, redimencionar y formatear las imagenes a formato de Tensores que permita ser interpretados por la red neuronal convolutional VGG19. Este paso se completa con la función de keras K.variable() que convierte definitivamente un numpy array en un Tensor.
</font>

In [0]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))        # Cargamos la imagen e ingresamos las medidas de altura y ancho
    img = img_to_array(img)                                               # Convierte los pixeles de la imagen a un array de numpy
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

*Respuesta:*
<font color="003300">
    
    
**deprocess_image()**

- Función que permite convertir un Tensor a un formato de imagen valido. Es la acción inversa de la realizada en la imagen anterior.
- Donde [103.939, 116.779, 123.68] son las medias de los canales de BGR para el proceso de VVG

</font>

In [0]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [0]:
                                                                      # get tensor representations of our images
                                                                      # K.variable convierte un numpy array en un tensor
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [0]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

In [0]:
                                                                      # k.placeholder --> incorporo la infromacion de img_rowa y img_ncols a un tensor
                                                                      # introduce los parametros de tamaño WH and C

print(combination_image)

Tensor("Placeholder:0", shape=(1, 400, 517, 3), dtype=float32)


*Aclaración:*
<font color="003300"><b>
**La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.**
</b></font>    

In [0]:
                                                                      # combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [0]:
                                                                      # build the VGG19 network with our 3 images as input
                                                                      # the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

                                                                      # get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])




80142336/80134624 [==============================] - 3s 0us/step






Model loaded.


# 4) En la siguientes celdas:

# - ¿Qué es la matriz de Gram?¿Para qué se usa? 

*Respuesta:*
<font color="003300">
- La Matriz de Gram, permite comprobar la interdependencia lineal, un conjunto de vectores será linealmente interdependientes si y solo si el determinante de Gram no es nulo(la matriz de Gram es invertible). 
- Es aplicado cuando se define la loss de estilo, ya que permite la extracción de los features maps de la imagen de estilo. Dado que las relaciones en las capas inferirores es predominantemente lineal.
- Esto último se logra al minimizar el cuadrado del error de la distancia entre la entrada de la matriz de Gram de la imagen original y la matriz de Gram de la imagen generada.
    
</font>


In [0]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

# Explicar qué mide cada una de las losses en las siguientes tres celdas.

*Respuesta:*


<font color="003300">
<ol>
    <li>style_loss(): Se utiliza para mantener el estilo de la imagen de referencia en la imagen generada.  En el estilo recogemos colores y patrones, por lo que se utilizan capas inferiores que capturan características de bajo nivel.
    </li>
    <li>content_loss(): Tiene por objeto mantener el contenido de la imagen base. Se considera la idea de que las imágenes con similar contorno tendrán representaciones similares en las capas superiores de la red.
    </li>
    <li>total_variation_loss(): Busca mantener la coherencia de la imagen base. Dado que el alfa y el beta representan los pesos del estilo y el contenido, la Loss Total, resulta en la representación del problema. Necesitamos que el contexto de la imagen resultante (identificación de objetos) sea similar a los ingresados/captados desde la imagen de contexto de entrada y el estilo entonces de la imagen resultante estará aportado por la imagen ingresada de estilo (colores y patrones). 
    </li>
</ol>
</font>

In [0]:
def style_loss(style, combination):                                 # Mantener el estilo de la imagen de referencia en la imagen generada
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [0]:
def content_loss(base, combination):                                # Tiene por objetivo mantener el contenido de la imagen base
    return K.sum(K.square(combination - base))


In [0]:
def total_variation_loss(x):
    assert K.ndim(x) == 4                                           # Si K.ndim == 4 no hace nada sino // Diseñada para matener la coherencia de la imagen base
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [0]:
                                                                    # Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [0]:
print(total_variation_loss(combination_image))
print(sl)
print(loss)

Tensor("Sum_7:0", shape=(), dtype=float32)
Tensor("truediv_4:0", shape=(), dtype=float32)
Tensor("add_7:0", shape=(), dtype=float32)


In [0]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

*Respuesta:*
<font color="003300">
- El proposito de las celdas (def eval_loss_and_grads(x) y class Evaluator()) **eval_loss_and_grads(x)** hace posible computar la loss y el gradiente en un paso  cuando estos son recibidos como dos funciones independientes. Esto se realiza porque scipy.optimize requiere las funciones de evaluación loss y gradiente por separado, pero realizar los cálculos en esta forma resultaría ineficiente.


- **from scipy.optimize import fmin_l_bfgs_b** La funcion fmin_l_bfgs_b minimiza la función evaluator.loss, con el gradiente evaluator.grads evaluando un máximo de 20 iteraciones.


- Para la síntesis de la imagen (Gatys, Ecker, Bethge) reemplazaron max-pooling por average pooling lo que mejora el flujo del gradiente y por el cual se obtienen resultados aparentemente mejores.

</font>




In [0]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3)) # la red espera una o mas imagenes como entrada
    outs = f_outputs([x])                       # Muestras, Filas, Columnas, Canales
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# Esta funcion class Evaluator() hace posible que se compute la loss y los gradientes en un solo paso
# mientras los recibe de dos funciones separadas. 'loss' y 'grads'.
# Esto se hace dado que scipy.optimize requiere las funciones loss y gradientes por separado
# y calcularlas se forma separada reduciria la performance.


In [0]:
# print(x.reshape((1, img_nrows, img_ncols, 3)))       le estamos diciendo a la VGG19 que le va a ir 1 imagen
#                                                      con esas dimenciones y 3 canales por lo colores

In [0]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [0]:
print(preprocess_image(base_image_path))

[[[[  98.061       85.221       72.32     ]
   [ 102.061       88.221       77.32     ]
   [ 105.061       91.221       80.32     ]
   ...
   [  34.060997    11.221001    -2.6800003]
   [  33.060997    10.221001    -3.6800003]
   [  32.060997     9.221001    -4.6800003]]

  [[  93.061       77.221       65.32     ]
   [  96.061       80.221       68.32     ]
   [  99.061       83.221       72.32     ]
   ...
   [  31.060997     8.221001    -5.6800003]
   [  30.060997     7.2210007   -6.6800003]
   [  28.060997     5.2210007   -8.68     ]]

  [[  93.061       77.221       65.32     ]
   [  96.061       80.221       68.32     ]
   [  98.061       82.221       70.32     ]
   ...
   [  31.060997     8.221001    -5.6800003]
   [  30.060997     7.2210007   -6.6800003]
   [  29.060997     6.2210007   -7.6800003]]

  ...

  [[  25.060997    35.221        6.3199997]
   [-103.939     -102.779     -123.68     ]
   [ -94.939      -90.779     -108.68     ]
   ...
   [ -99.939      -49.779      -58.

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [0]:
evaluator = Evaluator()
# Aplica el optimizador (L-BFGS) de scipy-based sobre los pixels de la imagen generada 
# para minimizar la funcion de perdida de la red neuronal correspondiente al estilo.

x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # Graba las imagenes generadas
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 7002775600.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 6s
Start of iteration 1
Current loss value: 4866963500.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 5s
Start of iteration 2
Current loss value: 4190534700.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 6s
Start of iteration 3
Current loss value: 3874594300.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 6s
Start of iteration 4
Current loss value: 3665689300.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 6s
Start of iteration 5
Current loss value: 3522369000.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 6s
Start of iteration 6
Current loss value: 3414872300.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 6s
Start of iteration 7
Curren

In [0]:
import shutil
archivo_salida = 'Resultado' + NombreSalida
shutil.make_archive(archivo_salida, 'zip', '/content/output')

'/content/Resultado_Loss_stylw_1_contw_10_TotVar_0.9.zip'

In [0]:
from google.colab import files
download_zip = '/content/'+ archivo_salida + '.zip'
files.download(download_zip) 

In [0]:
!rm -rf /content/output/

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

*Respuesta:* 

<font color="003300"><b>
*A continuación se exponen las imagenes que se originaron secuencialmente durante el entrenamiento, puede apreciarse como lentamente van absorbiendo la transferencia de estilo.*
</b></font>
    
    
| Parametros: style_weight = 10, content_weight, = 1 total_variation_weight = 0.9 | Parametros: style_weight = 10, content_weight, = 1 total_variation_weight = 0.0        
|:-: | :-:
| <img src="image/Resultado_Loss_stylw_10_contw_1_TotVar_0.9_.gif">| <img src="image/Resultado_Loss_stylw_10_contw_1_TotVar_0.0_.gif">



| Parametros: style_weight = 1, content_weight, = 10 total_variation_weight = 0.0 | Parametros: style_weight = 1, content_weight, = 10 total_variation_weight = 0.9        
|:-: | :-:
| <img src="image/Resultado_Loss_stylw_1_contw_10_TotVar_0.0_.gif">| <img src="image/Resultado_Loss_stylw_1_contw_10_TotVar_0.9_.gif">



| Parametros: style_weight = 5, content_weight, = 5 total_variation_weight = 0.0 | Parametros: style_weight = 5, content_weight, = 5 total_variation_weight = 0.9        
|:-: | :-:
| <img src="image/Resultado_Loss_stylw_5_contw_5_TotVar_0.0_.gif">| <img src="image/Resultado_Loss_stylw_5_contw_5_TotVar_0.9_.gif">






# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

*Respuesta:* 

<font color="003300"><b>
**El resultado obtenido del entrenamiento fue compilado en archivo gif animado con el proposito de visulizar los cambios en la imagen durante el entrenamiento.**
</b></font>

| Parametros: style_weight = 70, content_weight, = 30 total_variation_weight = 4 |
|:-: |
| <img src="image/Resultado2_Loss_stylw_70_contw_30_TotVar_4.gif">|

In [0]:
"""!rm /content/output/output_at_iteration_0.png
!rm /content/output/output_at_iteration_1.png
!rm /content/output/output_at_iteration_2.png
!rm /content/output/output_at_iteration_3.png
!rm /content/output/output_at_iteration_4.png
!rm /content/output/output_at_iteration_5.png
!rm /content/output/output_at_iteration_6.png
!rm /content/output/output_at_iteration_7.png
!rm /content/output/output_at_iteration_8.png
!rm /content/output/output_at_iteration_9.png
!rm /content/output/output_at_iteration_10.png
!rm /content/output/output_at_iteration_11.png
!rm /content/output/output_at_iteration_12.png
!rm /content/output/output_at_iteration_13.png
!rm /content/output/output_at_iteration_14.png
!rm /content/output/output_at_iteration_15.png
!rm /content/output/output_at_iteration_16.png
!rm /content/output/output_at_iteration_17.png
!rm /content/output/output_at_iteration_18.png
!rm /content/output/output_at_iteration_19.png
!rm /content/output/output_at_iteration_20.png
!rm /content/output/output_at_iteration_21.png
!rm /content/output/output_at_iteration_22.png
!rm /content/output/output_at_iteration_23.png
!rm /content/output/output_at_iteration_24.png
!rm /content/output/output_at_iteration_25.png
!rm /content/output/output_at_iteration_26.png
!rm /content/output/output_at_iteration_27.png
!rm /content/output/output_at_iteration_28.png
!rm /content/output/output_at_iteration_29.png
!rm /content/output/output_at_iteration_30.png
!rm /content/output/output_at_iteration_31.png
!rm /content/output/output_at_iteration_32.png
!rm /content/output/output_at_iteration_33.png
!rm /content/output/output_at_iteration_34.png
!rm /content/output/output_at_iteration_35.png
!rm /content/output/output_at_iteration_36.png
!rm /content/output/output_at_iteration_37.png
!rm /content/output/output_at_iteration_38.png
!rm /content/output/output_at_iteration_39.png
"""

In [0]:
"""for i in range(1,40,1):
  filetodelete = '/content/output/output_at_iteration_'+str(i)+'.png'
  #!rm [filetodelete]
  print(filetodelete)"""

"for i in range(1,40,1):\n  filetodelete = '/content/output/output_at_iteration_'+str(i)+'.png'\n  #!rm [filetodelete]\n  print(filetodelete)"